# Generate nice looking figures for the poster

In [1]:
from pathlib import Path
from IPython.display import Image
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import pandas as pd
import xarray as xr

Define the output path for all images.

In [2]:
outfolder = Path("../../figures")
outfolder.mkdir(exist_ok=True)
datafolder = Path("../../data")

## Synthetic data

In [3]:
%%bash -s $outfolder $datafolder

grid=$2/synthetic-san-andreas-2017.nc
diff=$2/synthetic-san-andreas-2017-3c-difference.nc
diff1c=$2/synthetic-san-andreas-2017-1c-difference.nc
samples=$2/synthetic-san-andreas-2017-samples.csv
vector_color="#333333"
basemap_args="-BNE -B2 --MAP_FRAME_TYPE=plain --MAP_FRAME_PEN=thicker,white --MAP_TICK_PEN=white --FONT_ANNOT=white"
cpt=synthetic.cpt
proj=M6i
region=-123/-115/33/38
table=synthetic.tmp

gmt begin
    gmt makecpt -Cpolar -T-1/1 -Q > $cpt
    
    gmt figure $1/synthetic-san-andreas-2017-sample pdf,PNG A
    gmt coast -R$region -J$proj -G"#666666" 
    gmt plot $samples -i3,4,2 -Sc0.2c -C$cpt
    tail -n +2 $samples | awk -F"," "{print \$4, \$5, atan2(\$1,\$2)*180/3.1415926, sqrt(\$1^2 + \$2^2)}" > $table
    gmt plot $table -SV0.15c+e -G$vector_color -W1.2p,$vector_color -i0,1,2,3+s0.03 
    gmt basemap $basemap_args
    gmt colorbar -DjBL+w1.75i/0.1i+h+o0.2/0.3i -C$cpt -Bx0.5 -By+l"mm/yr" -I --MAP_TICK_PEN=white --FONT_ANNOT=white --MAP_FRAME_PEN=thinner,white
    echo "122.2W 33.75N 90 20" | gmt plot -SV0.2c+e+jc -Gwhite -W1.2p,white --MAP_VECTOR_SHAPE=0.2 -i0,1,2,3+s0.03
    echo "121.5W 33.6N 20 mm/yr" | gmt text -F+f12p,white+jCB -D0/0.07i

    gmt figure $1/synthetic-san-andreas-2017 pdf,PNG A
    gmt grdimage $grid?up_velocity -R$region -J$proj -C$cpt -Q -I+n2
    gmt grdvector $grid?east_velocity $grid?north_velocity -S200 -G$vector_color -W0.8p,$vector_color -Ix8 -Q0.12c+e
    gmt basemap $basemap_args
    
    gmt figure $1/synthetic-san-andreas-2017-3c-diff pdf,PNG A
    gmt grdimage $diff?up_velocity -R$region -J$proj -C$cpt -Q -I+n2
    gmt grdvector $diff?east_velocity $diff?north_velocity -S200 -G$vector_color -W0.8p,$vector_color -Ix3 -Q0.0c+e
    gmt basemap $basemap_args
    
    gmt figure $1/synthetic-san-andreas-2017-1c-diff pdf,PNG A
    gmt grdimage $diff1c?up_velocity -R$region -J$proj -C$cpt -Q -I+n2
    gmt grdvector $diff1c?east_velocity $diff1c?north_velocity -S200 -G$vector_color -W0.8p,$vector_color -Ix3 -Q0.0c+e
    gmt basemap $basemap_args
gmt end

rm $cpt $table

du -h $1/synthetic-*

200K	../../figures/synthetic-san-andreas-2017-1c-diff.pdf
320K	../../figures/synthetic-san-andreas-2017-1c-diff.png
224K	../../figures/synthetic-san-andreas-2017-3c-diff.pdf
324K	../../figures/synthetic-san-andreas-2017-3c-diff.png
96K	../../figures/synthetic-san-andreas-2017.pdf
552K	../../figures/synthetic-san-andreas-2017.png
96K	../../figures/synthetic-san-andreas-2017-sample.pdf
288K	../../figures/synthetic-san-andreas-2017-sample.png


gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/m

In [4]:
Image(url=str(outfolder / "synthetic-san-andreas-2017-sample.png"), width=500)

In [5]:
Image(url=str(outfolder / "synthetic-san-andreas-2017-3c-diff.png"), width=500)

In [6]:
Image(url=str(outfolder / "synthetic-san-andreas-2017-1c-diff.png"), width=500)

In [7]:
Image(url=str(outfolder / "synthetic-san-andreas-2017.png"), width=500)

## PBO data

In [8]:
%%bash -s $outfolder $datafolder

data=$2/pbo-nam08-us-west-coast.csv
grid3c=$2/pbo-nam08-3c-spline-grid.nc
grid1c=$2/pbo-nam08-1c-spline-grid.nc
vector_color="#333333"
basemap_args="-BNE -B2 --MAP_FRAME_TYPE=plain --MAP_FRAME_PEN=thicker"
cpt=pbo.cpt
proj=M6i
region=-125/-115/32.5/43.5
table=pbo.tmp
figname=$1/pbo-nam08
figfmt="pdf,PNG A"

gmt begin
    gmt makecpt -Cpolar -T-8/8 -Q > $cpt
    
    gmt figure $figname-data $figfmt
    gmt coast -R$region -J$proj -G"#666666" -A10/0/1
    gmt plot $data -i0,1,4 -Sc0.2c -C$cpt
    gmt colorbar -DjBL+e+w1.75i/0.1i+h+o1/0.5i -C$cpt -Bx2 -By+l"mm/yr" --MAP_FRAME_PEN=thinner -I
    tail -n +2 $data | awk -F"," "{print \$1, \$2, atan2(\$3, \$4)*180/3.1415926, sqrt(\$3^2 + \$4^2)}" > $table
    gmt plot $table -SV0.1c+e -G$vector_color -W0.8p,$vector_color -i0,1,2,3+s0.03 
    echo "123.5W 33.75N 90 20" | gmt plot -SV0.2c+e+jc -Gblack -W1.2p,black --MAP_VECTOR_SHAPE=0.2 -i0,1,2,3+s0.03
    echo "122.5W 33.6N 20 mm/yr" | gmt text -F+f12p+jCB -D0/0.07i
    gmt basemap $basemap_args
    
    gmt figure $figname-3c $figfmt
    gmt grdimage $grid3c?up_velocity -R$region -J$proj -C$cpt -Q -I+n1
    gmt grdvector $grid3c?east_velocity $grid3c?north_velocity -S200 -G$vector_color -W0.8p,$vector_color -Ix2 -Q0.1c+e
    gmt coast -W1/1p,"#666666"
    gmt basemap $basemap_args
    
    
    gmt figure $figname-1c $figfmt
    gmt grdimage $grid1c?up_velocity -R$region -J$proj -C$cpt -Q -I+n1
    gmt grdvector $grid1c?east_velocity $grid1c?north_velocity -S200 -G$vector_color -W0.8p,$vector_color -Ix2 -Q0.1c+e
    gmt coast -W1/1p,"#666666"
    gmt basemap $basemap_args

gmt end

rm $cpt $table

du -h $1/pbo-*

112K	../../figures/pbo-nam08-1c.pdf
556K	../../figures/pbo-nam08-1c.png
112K	../../figures/pbo-nam08-3c.pdf
576K	../../figures/pbo-nam08-3c.png
136K	../../figures/pbo-nam08-data.pdf
472K	../../figures/pbo-nam08-data.png


gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/m

In [9]:
Image(url=str(outfolder / "pbo-nam08-3c.png"), width=500)

In [10]:
Image(url=str(outfolder / "pbo-nam08-1c.png"), width=500)

In [11]:
Image(url=str(outfolder / "pbo-nam08-data.png"), width=500)

## Equations


$$
\begin{bmatrix}
G_{ee} & G_{en} & \alpha\ G_{ev} \\
G_{ne} & G_{nn} & \alpha\ G_{nv} \\
\alpha\ G_{ve} & \alpha\ G_{vn} & G_{vv} \\
\end{bmatrix}
\begin{bmatrix}
f_e \\
f_n \\
f_v \\
\end{bmatrix}
=
\begin{bmatrix}
d_e \\
d_n \\
d_v \\
\end{bmatrix}
$$

## Background image for the poster

In [3]:
background = outfolder / "poster_background.jpg"

In [6]:
%%bash -s $background
gmt begin
gmt figure $1 jpg E120,A                                                                         
gmt grdimage @earth_relief_01m -R-195/-126/10/50 -JM72i -Cocean -I+a60+nt0.5            
gmt end
du -h $1

6.6M	../../figures/poster_background.jpg


gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)
gmt: /home/leo/miniconda3/lib/libcurl.so.4: no version information available (required by /home/leo/pkg/lib64/libgmt.so.6)


In [7]:
Image(url=str(background), width=800, embed=False)